In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import calendar
import numpy as np
calendar.setfirstweekday(0)

def get_week_of_month(year, month, day):
    x = np.array(calendar.monthcalendar(year, month))
    week_of_month = np.where(x==day)[0][0] + 1
    return(week_of_month)

get_week_of_month(2021,11,30)

5

In [3]:
import datetime

In [4]:
from datetime import datetime
import calendar
my_date = datetime.strptime('2021-11-30', "%Y-%m-%d")
calendar.day_name[my_date.weekday()]

'Tuesday'

In [5]:
def return_day(date_string):
    my_date = datetime.strptime(date_string, "%Y-%m-%d")
    return calendar.day_name[my_date.weekday()]

In [6]:
return_day('2021-11-30')

'Tuesday'

In [465]:
train = pd.read_csv("train.csv")
print('Shape of the train data:',train.shape)
print(train.head(10))

Shape of the train data: (913000, 4)
         date  store  item  sales
0  2013-01-01      1     1     13
1  2013-01-02      1     1     11
2  2013-01-03      1     1     14
3  2013-01-04      1     1     13
4  2013-01-05      1     1     10
5  2013-01-06      1     1     12
6  2013-01-07      1     1     10
7  2013-01-08      1     1      9
8  2013-01-09      1     1     12
9  2013-01-10      1     1      9


In [466]:
train['year']=train['date'].apply(lambda x: x[:4]).astype(int)
train['month']=train['date'].apply(lambda x: x[5:7]).astype(int)
train['day']=train['date'].apply(lambda x: x[8:10]).astype(int)

In [467]:
train

,date,store,item,sales,year,month,day
0,2013-01-01,1,1,13,2013,1,1
1,2013-01-02,1,1,11,2013,1,2
2,2013-01-03,1,1,14,2013,1,3
3,2013-01-04,1,1,13,2013,1,4
4,2013-01-05,1,1,10,2013,1,5
...,...,...,...,...,...,...,...
912995,2017-12-27,10,50,63,2017,12,27
912996,2017-12-28,10,50,59,2017,12,28
912997,2017-12-29,10,50,74,2017,12,29
912998,2017-12-30,10,50,62,2017,12,30


In [468]:
train['day_of_week']=train['date'].apply(return_day)

In [469]:
train['week_number']=train.apply(lambda x: get_week_of_month(x.year, x.month,x.day), axis=1)

In [470]:
train['date']=pd.to_datetime(train['date'])

In [471]:
train["is_wknd"] = train.date.dt.weekday // 4

In [472]:
train['is_month_start'] = train.date.dt.is_month_start.astype(int)
train['is_month_end'] = train.date.dt.is_month_end.astype(int)

In [473]:
train['day_of_year'] = train.date.dt.dayofyear
train['week_of_year'] = train.date.dt.weekofyear
train['day_of_year'] = train.date.dt.dayofyear

C:\Users\Jordan Tan Rei Yao\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [474]:
train['quarter'] = train.date.dt.quarter

In [475]:
train.drop(['day'] ,axis=1, inplace=True)

In [476]:
from sklearn.preprocessing import OneHotEncoder

In [477]:
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))

In [478]:
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe


OH_X_train = lag_features(train, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])

In [479]:
OH_X_train

,date,store,item,sales,year,month,day_of_week,week_number,is_wknd,is_month_start,...,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,2013-01-01,1,1,13,2013,1,Tuesday,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11,2013,1,Wednesday,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14,2013,1,Thursday,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13,2013,1,Friday,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10,2013,1,Saturday,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,2017-12-27,10,50,63,2017,12,Wednesday,5,0,0,...,79.500915,70.589363,83.196524,79.028712,79.100628,106.213969,90.564911,44.218616,78.114782,53.167525
912996,2017-12-28,10,50,59,2017,12,Thursday,5,0,0,...,79.155696,88.231375,87.791939,76.538136,87.611172,87.891890,124.590364,66.896236,89.021621,42.947500
912997,2017-12-29,10,50,74,2017,12,Friday,5,1,0,...,90.801913,99.445664,97.629425,100.264610,98.002535,99.716919,113.731077,63.622318,100.164049,59.846363
912998,2017-12-30,10,50,62,2017,12,Saturday,5,1,0,...,103.027153,96.353472,97.567293,92.263051,91.677201,90.875961,119.716286,66.392587,113.043339,61.586423


In [480]:
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store", "item"])['sales']. \
            transform(lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + \
            random_noise(dataframe)
    return dataframe

In [481]:
OH_X_train = roll_mean_features(OH_X_train, [182,365, 546])

In [482]:
OH_X_train

,date,store,item,sales,year,month,day_of_week,week_number,is_wknd,is_month_start,...,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_roll_mean_182,sales_roll_mean_365,sales_roll_mean_546
0,2013-01-01,1,1,13,2013,1,Tuesday,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11,2013,1,Wednesday,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14,2013,1,Thursday,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13,2013,1,Friday,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10,2013,1,Saturday,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,2017-12-27,10,50,63,2017,12,Wednesday,5,0,0,...,79.028712,79.100628,106.213969,90.564911,44.218616,78.114782,53.167525,84.336462,88.183457,81.839769
912996,2017-12-28,10,50,59,2017,12,Thursday,5,0,0,...,76.538136,87.611172,87.891890,124.590364,66.896236,89.021621,42.947500,87.592998,91.253246,79.497826
912997,2017-12-29,10,50,74,2017,12,Friday,5,1,0,...,100.264610,98.002535,99.716919,113.731077,63.622318,100.164049,59.846363,87.349422,89.185022,81.534793
912998,2017-12-30,10,50,62,2017,12,Saturday,5,1,0,...,92.263051,91.677201,90.875961,119.716286,66.392587,113.043339,61.586423,84.748841,91.223375,80.337434


In [483]:
def exponential_weighted_average(data, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            data['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                data.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return data
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]
OH_X_train = exponential_weighted_average(OH_X_train, alphas, lags)

In [485]:
OH_X_train= pd.get_dummies(OH_X_train, columns=['store', 'item', 'day_of_week','week_number','month'])

In [ ]:
OH_X_train = OH_X_train.reindex(sorted(OH_X_train.columns), axis=1)

In [491]:
y=OH_X_train['sales']

In [492]:
y1_train=np.log1p(y)

In [493]:
OH_X_train.drop(['sales'],axis=1,inplace=True)

In [494]:
OH_X_train.drop(['year'],axis=1,inplace=True)

In [495]:
OH_X_train.drop(['date'],axis=1,inplace=True)

In [498]:
OH_X_train.columns

Index(['day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday',
       'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday',
       'day_of_week_Wednesday', 'day_of_year', 'is_month_end',
       'is_month_start',
       ...
       'store_7', 'store_8', 'store_9', 'week_number_1', 'week_number_2',
       'week_number_3', 'week_number_4', 'week_number_5', 'week_number_6',
       'week_of_year'],
      dtype='object', length=149)

In [500]:
# Custom Cost Function
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

In [502]:
import lightgbm as lgb

In [503]:
# LightGBM model parameters

model_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 20000,
               'feature_fraction': 0.9,                                                                                                              
               'bagging_fraction': 0.8,                                                                                                              
               'bagging_freq': 5,                                                                                                                    
               'lambda_l1': 0.06,                                                                                                                    
               'lambda_l2': 0.05,
              'nthread': -1}

"\nlgb_params = {                                                                                                                            \n        'nthread': -1,\n        'metric': 'mae',\n        'boosting_type': 'gbdt',    \n        'max_depth': 7,\n        'num_leaves': 28,   \n        'task': 'train',                                                                                                                      \n        'objective': 'regression_l1',                                                                                                         \n        'learning_rate': 0.05,                                                                                                                \n        'feature_fraction': 0.9,                                                                                                              \n        'bagging_fraction': 0.8,                                                                                                          

In [504]:
lgbtrain = lgb.Dataset(data=OH_X_train, label=y1_train, feature_name=list(OH_X_train.columns))

In [505]:
lgbval = lgb.Dataset(data=val, label=y1_val, reference=lgbtrain, feature_name=list(OH_X_train.columns))

In [506]:
model = lgb.train(model_params, lgbtrain,
                  num_boost_round=model_params['num_boost_round'],
                  feval=lgbm_smape,
                  verbose_eval=100)

C:\Users\Jordan Tan Rei Yao\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\Jordan Tan Rei Yao\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.295920 seconds.
You can set `force_col_wise=true` to remove the overhead.


In [507]:
### START OF PREDICTING ON TEST DATA

In [508]:
test = pd.read_csv("test.csv")

In [509]:
train2 = pd.read_csv("train.csv")

In [510]:
test=pd.concat([train2,test],sort=False)

In [511]:
test

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN
...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0
44996,2018-03-28,10,50,NaN,44996.0
44997,2018-03-29,10,50,NaN,44997.0
44998,2018-03-30,10,50,NaN,44998.0


In [512]:
test['year']=test['date'].apply(lambda x: x[:4]).astype(int)
test['month']=test['date'].apply(lambda x: x[5:7]).astype(int)
test['day']=test['date'].apply(lambda x: x[8:10]).astype(int)

In [513]:
test

,date,store,item,sales,id,year,month,day
0,2013-01-01,1,1,13.0,NaN,2013,1,1
1,2013-01-02,1,1,11.0,NaN,2013,1,2
2,2013-01-03,1,1,14.0,NaN,2013,1,3
3,2013-01-04,1,1,13.0,NaN,2013,1,4
4,2013-01-05,1,1,10.0,NaN,2013,1,5
...,...,...,...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0,2018,3,27
44996,2018-03-28,10,50,NaN,44996.0,2018,3,28
44997,2018-03-29,10,50,NaN,44997.0,2018,3,29
44998,2018-03-30,10,50,NaN,44998.0,2018,3,30


In [514]:
test['day_of_week']=test['date'].apply(return_day)

In [515]:
test['week_number']=test.apply(lambda x: get_week_of_month(x.year, x.month,x.day), axis=1)

In [516]:
test

,date,store,item,sales,id,year,month,day,day_of_week,week_number
0,2013-01-01,1,1,13.0,NaN,2013,1,1,Tuesday,1
1,2013-01-02,1,1,11.0,NaN,2013,1,2,Wednesday,1
2,2013-01-03,1,1,14.0,NaN,2013,1,3,Thursday,1
3,2013-01-04,1,1,13.0,NaN,2013,1,4,Friday,1
4,2013-01-05,1,1,10.0,NaN,2013,1,5,Saturday,1
...,...,...,...,...,...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0,2018,3,27,Tuesday,5
44996,2018-03-28,10,50,NaN,44996.0,2018,3,28,Wednesday,5
44997,2018-03-29,10,50,NaN,44997.0,2018,3,29,Thursday,5
44998,2018-03-30,10,50,NaN,44998.0,2018,3,30,Friday,5


In [517]:
test['date']=pd.to_datetime(test['date'])

In [518]:
test["is_wknd"] = test.date.dt.weekday // 4

In [519]:
test['is_month_start'] = test.date.dt.is_month_start.astype(int)
test['is_month_end'] = test.date.dt.is_month_end.astype(int)

In [520]:
test['day_of_year'] = test.date.dt.dayofyear
test['week_of_year'] = test.date.dt.weekofyear
test['day_of_year'] = test.date.dt.dayofyear

C:\Users\Jordan Tan Rei Yao\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [521]:
test['quarter'] = test.date.dt.quarter

In [522]:
test.drop(['day','date','id'] ,axis=1, inplace=True)

In [523]:
OH_X_test = lag_features(test, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])

In [524]:
OH_X_test = roll_mean_features(OH_X_test, [182,365, 546])

In [525]:
OH_X_test = exponential_weighted_average(OH_X_test, alphas, lags)

In [527]:
OH_X_test= pd.get_dummies(OH_X_test, columns=['store', 'item', 'day_of_week','week_number','month'])

In [528]:
OH_X_test = OH_X_test.loc[OH_X_test.sales.isna()]

In [530]:
OH_X_test.drop(['sales'],axis=1,inplace=True)

In [531]:
OH_X_test = OH_X_test.reindex(sorted(OH_X_test.columns), axis=1)

In [534]:
OH_X_test.drop(['year'],axis=1,inplace=True)

In [535]:
test_pred=model.predict(OH_X_test, num_iteration=model.best_iteration)

In [537]:
submission=pd.read_csv("test.csv")

In [538]:
predicted=pd.DataFrame({'sales':np.expm1(test_pred)})

In [539]:
submission= pd.concat([submission, predicted ], axis=1)

In [541]:
submission[['id','sales']].to_csv('submission_LightGBM.csv', index = False)